In [9]:
from sklearn import svm, datasets
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
iris = datasets.load_iris()

In [17]:
X = iris.data
print(X.shape)

# Add noisy features
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]
print(X.shape)

(150, 4)
(150, 804)


In [13]:
y = iris.target
y.shape

(150,)

In [23]:
y = label_binarize(y, classes=[0, 1, 2])
y.shape

(150, 3)

In [28]:
y[:, 2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [29]:
from sklearn.metrics import precision_recall_curve

In [35]:
pre, rec, ths = precision_recall_curve(
    y[:, 0],
    random_state.randn(y.shape[0])
)

In [37]:
ths.shape

(138,)

In [58]:
from sklearn.preprocessing import MultiLabelBinarizer

In [61]:
mlb = MultiLabelBinarizer(classes=[1, 2, 3])
mlb.fit_transform([(1,), (1, 2), (1,), (2,), (1, 2)])

array([[1, 0, 0],
       [1, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 1, 0]])

In [60]:
mlb.classes_

array([1, 2])

In [64]:
precision_score(
    [[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1]],
    [[1, 0, 0], [1, 1, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]],
    average='micro'
) # OK

0.2857142857142857

In [30]:
precision_recall_fscore_support(
    [[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1]],
    [[1, 0, 0], [1, 1, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]],
    average='micro'
)

(0.2857142857142857, 0.4, 0.3333333333333333, None)

In [67]:
gt = np.array([[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1]])
ps = np.array([[0.5, 0, 0], [0.7, 0.2, 0.1], [0.1, 0.8, 0.2], [0.78, 0, 0], [0.1, 0.1, 0.6]])

precision_recall_curve(
    gt[:, 0],
    ps[:, 0],
) # OK

(array([1., 1., 1., 1.]),
 array([1.        , 0.66666667, 0.33333333, 0.        ]),
 array([0.5 , 0.7 , 0.78]))

In [69]:
gt.ravel()

array([1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1])

In [70]:
precision_recall_curve(
    gt.ravel(),
    ps.ravel()
) # OK

(array([1., 1., 1., 1., 1., 1.]),
 array([1. , 0.8, 0.6, 0.4, 0.2, 0. ]),
 array([0.5 , 0.6 , 0.7 , 0.78, 0.8 ]))

In [1]:
### LOAD PREDICTIONS
import pickle

In [74]:
GROUND_TRUTH = "ground-truth"
PREDICTION = "prediction"
SEQ_LENGTH = "seq-length"


def default_value():
    return {
        GROUND_TRUTH: [],
        PREDICTION: [],
        SEQ_LENGTH: 0
    }

pred = pickle.load(open('../data/predictions_2021-05-16T12:22:03.722505.p', 'rb'))

In [10]:
# Goal
#   - 10 thresholdovu evaluaciu
#       --> podla thresholdu urobit binarizaciu --> vypocitat 'micro' Precision/Recall/F1-Score
#       --> reportnut tieto hodnoty do tensorboardu --> dostanem 10 threshold grafov
#   - vypocet accuracy pre Classification rovnakym sposobom
#   - vypocitat AP pre detection a dat do grafu
#   - precision-recall-curve ktory ulozim ako PNG

#   - kumulativna podla thresholdu ??
#       - viem to vybrat z dat pre Precision/Recall/F1-Score z thresholdov

In [75]:
# 10 Threshold evaluaction
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_fscore_support, average_precision_score

In [87]:
y_cont = []
x_cont = []
for _, values in pred.items():
    y_cont += values[GROUND_TRUTH]
    x_cont += [t[0] for t in values[PREDICTION]]
    
y_cont = np.array(y_cont)
x_cont = np.array(x_cont)

In [89]:
x_cont.shape

(272166, 43)

In [90]:
# ground-truth binarization
y_bin = label_binarize(y_cont, classes=list(range(0, 43)))

In [96]:
# calculate Precision, Recall, F1 Score
for t in np.arange(0.1, 1.0, 0.1):
    x_bin = np.where(x_cont > t, 1, 0)
    res = precision_recall_fscore_support(
        y_bin,
        x_bin,
        average='micro',
        zero_division=0
    )
    res = [round(v, 3) for v in res if v is not None]
    print(f"{round(t, 1)}, Precision: {res[0]}, Recall: {res[1]}, F1-Score: {res[2]}")

0.1, Precision: 0.318, Recall: 0.589, F1-Score: 0.413
0.2, Precision: 0.432, Recall: 0.489, F1-Score: 0.459
0.3, Precision: 0.515, Recall: 0.421, F1-Score: 0.463
0.4, Precision: 0.591, Recall: 0.367, F1-Score: 0.453
0.5, Precision: 0.663, Recall: 0.319, F1-Score: 0.431
0.6, Precision: 0.728, Recall: 0.274, F1-Score: 0.398
0.7, Precision: 0.79, Recall: 0.228, F1-Score: 0.354
0.8, Precision: 0.855, Recall: 0.166, F1-Score: 0.278
0.9, Precision: 0.942, Recall: 0.078, F1-Score: 0.144


In [93]:
ap_score = average_precision_score(
    y_bin,
    x_cont,
    average='macro'
)
print(f'\nAP: {ap_score}')


AP: 0.49069291293506195
